In [1]:
import geopandas as gpd
import pandas as pd

## Open 2023 Population Estimates

In [86]:
county_populations = pd.read_csv(
    "data/co-est2023-alldata.csv", sep=",", encoding="latin-1"
)

In [96]:
county_populations["FIPS"] = county_populations.apply(
    lambda row: f"{row['STATE']:02d}{row['COUNTY']:03d}", axis=1
)

In [109]:
conn_df = county_populations[county_populations["STATE"] == 9]

In [110]:
conn_df

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,...,RINTERNATIONALMIG2021,RINTERNATIONALMIG2022,RINTERNATIONALMIG2023,RDOMESTICMIG2021,RDOMESTICMIG2022,RDOMESTICMIG2023,RNETMIG2021,RNETMIG2022,RNETMIG2023,FIPS
315,40,1,1,9,0,Connecticut,Connecticut,3605912,3577586,3603691,...,1.731447,4.315348,4.224813,5.300450,-2.795465,-2.495474,7.031897,1.519883,1.729339,09000
316,50,1,1,9,110,Connecticut,Capitol Planning Region,976275,962436,970142,...,1.756203,4.580349,4.461342,6.156543,-1.956534,-1.392629,7.912747,2.623815,3.068713,09110
317,50,1,1,9,120,Connecticut,Greater Bridgeport Planning Region,325774,324387,326752,...,2.770530,6.475224,6.338045,1.695491,-7.961742,-7.813359,4.466020,-1.486518,-1.475314,09120
318,50,1,1,9,130,Connecticut,Lower Connecticut River Valley Planning Region,174225,173268,175430,...,0.745631,2.179778,2.089740,14.253021,3.528622,0.919940,14.998652,5.708399,3.009679,09130
319,50,1,1,9,140,Connecticut,Naugatuck Valley Planning Region,450435,449285,452281,...,1.144675,3.152767,3.011922,5.434988,0.432733,2.233659,6.579662,3.585499,5.245582,09140
320,50,1,1,9,150,Connecticut,Northeastern Connecticut Planning Region,95336,95222,95605,...,0.282979,1.063869,1.006015,6.225534,6.675254,6.544285,6.508513,7.739123,7.550301,09150
321,50,1,1,9,160,Connecticut,Northwest Hills Planning Region,112465,112202,112953,...,0.621794,1.980286,1.887625,8.838356,4.243469,2.196348,9.460150,6.223755,4.083973,09160
322,50,1,1,9,170,Connecticut,South Central Connecticut Planning Region,570455,564076,568450,...,1.767730,3.931910,3.952277,6.196767,-3.476266,-4.789892,7.964497,0.455644,-0.837615,09170
323,50,1,1,9,180,Connecticut,Southeastern Connecticut Planning Region,280428,278342,278820,...,1.220471,2.143073,2.086274,1.949164,1.472915,0.561827,3.169635,3.615988,2.648101,09180
324,50,1,1,9,190,Connecticut,Western Connecticut Planning Region,620519,618368,623258,...,2.470953,6.463288,6.363674,3.149096,-9.548075,-7.371082,5.620050,-3.084788,-1.007408,09190


In [100]:
conn_gdf = county_populations_2023_gdf[county_populations_2023_gdf["STATEFP"] == "09"][
    ["STATEFP", "FIPS"]
]

In [106]:
conn_gdf["FIPS"]

56      09009
659     09001
660     09007
661     09011
1238    09005
1239    09013
1991    09015
2003    09003
Name: FIPS, dtype: object

In [108]:
conn_df["FIPS"]

315    09000
316    09110
317    09120
318    09130
319    09140
320    09150
321    09160
322    09170
323    09180
324    09190
Name: FIPS, dtype: object

In [103]:
conn_df.merge(conn_gdf, on="FIPS", how="right")

,STNAME,FIPS,STATEFP
0,NaN,09009,09
1,NaN,09001,09
2,NaN,09007,09
3,NaN,09011,09
4,NaN,09005,09
5,NaN,09013,09
6,NaN,09015,09
7,NaN,09003,09


In [46]:
county_populations_2023_df = county_populations[["POPESTIMATE2023", "FIPS"]].dropna()

## Open County Data

In [47]:
file_path = "data/cb_2018_us_county_500k/cb_2018_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

In [52]:
counties_gdf = counties_gdf.rename(columns={"GEOID": "FIPS"})

In [70]:
county_populations_2023_gdf = counties_gdf.merge(
    county_populations_2023_df, on="FIPS", how="left"
)

## Get McDonalds Data

In [71]:
mcdonalds_gdf = gpd.read_file("data/mcdonalds_va_h3_5.gpkg")

## Intersect Data

In [78]:
county_populations_2023_gdf = county_populations_2023_gdf.to_crs(mcdonalds_gdf.crs.srs)

In [85]:
county_populations_2023_gdf[county_populations_2023_gdf["STATEFP"] == "09"]

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,FIPS,NAME,LSAD,ALAND,AWATER,geometry,POPESTIMATE2023
56,09,009,00212798,0500000US09009,09009,New Haven,06,1565688367,667081242,"MULTIPOLYGON (((-72.76143 41.24233, -72.75973 ...",NaN
659,09,001,00212794,0500000US09001,09001,Fairfield,06,1618651428,549293518,"MULTIPOLYGON (((-73.21717 41.14391, -73.21611 ...",NaN
660,09,007,00212797,0500000US09007,09007,Middlesex,06,956493754,180679140,"POLYGON ((-72.75294 41.5925, -72.74284 41.5944...",NaN
661,09,011,00212799,0500000US09011,09011,New London,06,1722716728,276657755,"POLYGON ((-72.46673 41.5839, -72.42886 41.5889...",NaN
1238,09,005,00212796,0500000US09005,09005,Litchfield,06,2384116952,62334525,"POLYGON ((-73.51795 41.67086, -73.51678 41.687...",NaN
1239,09,013,00212668,0500000US09013,09013,Tolland,06,1062807467,17549693,"POLYGON ((-72.51733 41.8699, -72.51692 41.8736...",NaN
1991,09,015,00212801,0500000US09015,09015,Windham,06,1328478475,21477921,"POLYGON ((-72.25208 41.72706, -72.25264 41.728...",NaN
2003,09,003,00212338,0500000US09003,09003,Hartford,06,1903543897,40543777,"POLYGON ((-73.02054 42.00009, -73.00876 42.038...",NaN


In [79]:
county_with_counts_gdf = county_populations_2023_gdf.join(
    gpd.sjoin(mcdonalds_gdf, county_populations_2023_gdf)
    .groupby("index_right")
    .size()
    .rename("stores"),
    how="left",
)

In [80]:
county_with_counts_gdf = county_with_counts_gdf.fillna(0)
county_with_counts_gdf["stores"] = county_with_counts_gdf["stores"].astype(int)
county_with_counts_gdf = county_with_counts_gdf[
    ["STATEFP", "geometry", "stores", "NAME", "POPESTIMATE2023"]
]

In [81]:
county_with_counts_gdf["per_capita"] = county_with_counts_gdf["POPESTIMATE2023"].div(
    county_with_counts_gdf["stores"]
)

In [32]:
county_with_counts_gdf.to_file("data/counties_per_capita.gpkg")